<img src="https://ucfai.org//course/sp19/conv-nets/banner.jpg">

<div class="col-12">
    <a class="btn btn-success btn-block" href="https://ucfai.org/signup">
        First Attendance? Sign Up!
    </a>
</div>

<div class="col-12">
    <h1> How Computers Can See and Other Ways Machines Can Think </h1>
    <hr>
</div>

<div style="line-height: 2em;">
    <p>by: 
        <strong> Irene Tanner</strong>
        (<a href="https://github.com/irenelt97">@irenelt97</a>)
    
        <strong> John Muchovej</strong>
        (<a href="https://github.com/ionlights">@ionlights</a>)
     on 2019-02-20</p>
</div>